# Convolutional Neural Network

## Import libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Part 1 - Data Processing

In [2]:
tf.__version__

'2.2.0'

## Preprocess training set - Image Augmentation

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'data/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


## Preprocess test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = train_datagen.flow_from_directory(
        'data/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 2000 images belonging to 2 classes.


# Part 2 - Build CNN

## Initialize CNN

In [5]:
cnn = tf.keras.models.Sequential()

## Step 1 - Convolution

In [6]:
#filter = feature detector's matrix size. (its just experimenting number of features..standard is like 32)
#colored input shape = 64 x 64 x 3 (3 --> RGB, 3 colors)
#black and white input shape = 64 x 64 x 1 (black or white)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))


## Step 2 - Pooling

In [7]:
#pool size matrix square size and strides is how many pixes we want to move or in other words, how the square matrix is traversing in the main feature map
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Adding a second Convolutional Layer

In [8]:
#filter = feature detector's matrix size. (its just experimenting number of features..standard is like 32)
#input_shape is same as original or first convolutional layer, and so remove it from here
#black and white input shape = 64 x 64 x 1 (black or white)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

## Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

## Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training The CNN

## Compile CNN

In [12]:
cnn.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics =['accuracy'])

## Train CNN on training set and evaluate with test set

In [26]:
checkpoint_path = "data/cp.ckpt"
# checkpoint_dir = os.getcwd()
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_best_only=True, 
                                                 save_weights_only=True,
                                                 verbose=1)

cnn.fit(x=training_set, validation_data=test_set, epochs=25, callbacks=[cp_callback])

Epoch 1/25
250/250 [==============================] - ETA: 0s - loss: 0.6827 - accuracy: 0.5565
Epoch 00001: val_loss improved from inf to 0.63358, saving model to data/cp.ckpt
250/250 [==============================] - 40s 160ms/step - loss: 0.6827 - accuracy: 0.5565 - val_loss: 0.6336 - val_accuracy: 0.6420
Epoch 2/25
250/250 [==============================] - ETA: 0s - loss: 0.6081 - accuracy: 0.6750
Epoch 00002: val_loss improved from 0.63358 to 0.57826, saving model to data/cp.ckpt
250/250 [==============================] - 32s 128ms/step - loss: 0.6081 - accuracy: 0.6750 - val_loss: 0.5783 - val_accuracy: 0.7075
Epoch 3/25
250/250 [==============================] - ETA: 0s - loss: 0.5631 - accuracy: 0.7074
Epoch 00003: val_loss improved from 0.57826 to 0.55282, saving model to data/cp.ckpt
250/250 [==============================] - 33s 131ms/step - loss: 0.5631 - accuracy: 0.7074 - val_loss: 0.5528 - val_accuracy: 0.7235
Epoch 4/25
250/250 [==============================] - ETA: 

# Part 4 - Making A Single Predicton

In [16]:
import numpy as np
from keras.preprocessing import image
single_test = image.load_img('data/dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
single_test = image.img_to_array(single_test)
single_test = np.expand_dims(single_test, axis=0)
result = cnn.predict(single_test)

# how to know 0 is cat and 1 is dog?
print(training_set.class_indices)

#first [0] is for batch size
#second [0] is prediction array size which is again just 1.
if (result[0][0] == 1):
    prediction = 'Dog'
else:
    prediction = 'Cat'

print (prediction)

{'cats': 0, 'dogs': 1}
Dog


In [16]:
import numpy as np
from keras.preprocessing import image

checkpoint_path = "data/cp.ckpt"
# checkpoint_dir = os.getcwd()
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_best_only=True, 
                                                 save_weights_only=True,
                                                 verbose=1)


single_test = image.load_img('data/dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
single_test = image.img_to_array(single_test)
single_test = np.expand_dims(single_test, axis=0)

cnn.load_weights(checkpoint_path)
result = cnn.predict(single_test)

# how to know 0 is cat and 1 is dog?
print(training_set.class_indices)

#first [0] is for batch size
#second [0] is prediction array size which is again just 1.
if (result[0][0] == 1):
    prediction = 'Dog'
else:
    prediction = 'Cat'

print (prediction)

{'cats': 0, 'dogs': 1}
Dog
